In [4]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

## Counting Cliques

In [23]:
def neighbors(G, n):
    return set(G[n]) if n in G else set()

In [65]:
from itertools import chain
def count_3_4_cliques(G, num_3_4_cliques, new_edge):
    """Appends the new edge to the previous edges and updates the prev_cliques structure accordingly.
    For a new edge (a,b), check for all nodes c such that (a,b) and (a,c) are already in the graph,
    and append those to the new Counter;
    Also check for all nodes c and d such that (a,c), (c,d), and (d,b) are in the graph, 
    and append those to the new Counter."""
    a, b, *data = new_edge
    #print(a,b)
    # 3 cliques
    adj_a = neighbors(G,a)
    adj_b = neighbors(G,b)
    
    all_c = adj_a & adj_b # intersection of neighbors of a and b
    num_3_4_cliques[3] += len(all_c)
    # 4 cliques
    """
    possible_d = set().union(*[neighbors(G,c) for c in all_c])
    all_d = possible_d & all_c
    print(all_c, "&", possible_d, "->", all_d)
    """
    all_d = [v for c in all_c for v in neighbors(G,c) if v in all_c]
    #print(all_d)
    
    num_3_4_cliques[4] += len(all_d)//2
    # return
    G.add_edge(a, b)
    return G, num_3_4_cliques

## Testing 4-cliques

In [36]:
def num_q_cliques(G, q):
    """Computes the number of q-cliques of a given graph (defined through a networkx graph)"""
    cliques = list(nx.enumerate_all_cliques(G))
    q_cliques = [c for c in cliques if len(c) == q]
    return len(q_cliques)

In [32]:
test_G = nx.complete_graph(10)

In [68]:
#nx.draw_networkx(test_G)

In [67]:
"""
from collections import Counter
E = list(test_G.edges())
num_3_4_cliques = Counter()
G_sub = nx.Graph()
u, v = list(E[0])
G_sub.add_edge(u, v)
q3, q4 = [], []

for k in range(1,len(E)):
    G_sub, num_3_4_cliques = count_3_4_cliques(G_sub, num_3_4_cliques, E[k])
    #q3.append(num_3_4_cliques[3])
    #q4.append(num_3_4_cliques[4])
    print(E[k], ":", num_q_cliques(G_sub, 4), "vs", num_3_4_cliques[4])
"""

'\nfrom collections import Counter\nE = list(test_G.edges())\nnum_3_4_cliques = Counter()\nG_sub = nx.Graph()\nu, v = list(E[0])\nG_sub.add_edge(u, v)\nq3, q4 = [], []\n\nfor k in range(1,len(E)):\n    G_sub, num_3_4_cliques = count_3_4_cliques(G_sub, num_3_4_cliques, E[k])\n    #q3.append(num_3_4_cliques[3])\n    #q4.append(num_3_4_cliques[4])\n    print(E[k], ":", num_q_cliques(G_sub, 4), "vs", num_3_4_cliques[4])\n'

In [69]:
#print(q4)

## Other Functions

In [ ]:
erdos_reyni_p_MLE = lambda num_edges, num_nodes: \
    (2 * num_edges) / (num_nodes * (num_nodes - 1))

In [ ]:
from math import comb
erdos_reyni_estimated_q_cliques = lambda q, num_nodes, p: \
    comb(num_nodes, q) * (p ** (q * (q - 1) / 2))

In [ ]:
def ssbm_a_b_CMM(num_nodes, num_edges, num_3_cliques):
    """Return the expected values for a and b for the SSBM with 2-communities SSBM(n,2,a,b) 
    by Modified Constrained Moment Matching Algorithm 2."""
    n, m, t = num_nodes, num_edges, num_3_cliques
    p = 2*m/(n*(n-1))
    delta = 6*t/(n*(n-1)*(n-2)) - p**3
    A1, A2 = max(0,2*p-1), min(1,2*p)
    P = lambda A:(A-p)**3 - delta 
    PA1, PA2 = P(A1), P(A2)
    if PA1 <= 0 <= PA2: a_CMM, b_CMM = p + delta**(1/3), p - delta**(1/3) 
    elif PA2 < 0:       a_CMM, b_CMM = A2, 2*p-A2
    else:               a_CMM, b_CMM = A1, 2*p-A1
    # Adjust to produce the Modified Constrained Moment Matching estimates
    if b_CMM == 0:   a_MCMM, b_MCMM = 0.99*a_CMM, 0.01*a_CMM
    elif a_CMM == 0: a_MCMM, b_MCMM = 0.01*b_CMM, 0.99*b_CMM
    else:            a_MCMM, b_MCMM = a_CMM, b_CMM
    return a_MCMM, b_MCMM

In [1]:
def ssbm_estimated_q_cliques(q, num_nodes, a, b):
    n = num_nodes
    if q == 3:
        expected_3_cliques = (
            (n*(n-1)*(n-2)/24) * (a**3 + 3*a*b**2)
        )
        return expected_3_cliques
    elif q == 4:
        E_n1 = n / 2
        E_n1_square = (n ** 2 + n) / 4
        E_n1_cube = (n ** 2) * (n + 3) / 8
        E_n1_quad = n * (n + 1) * (n ** 2 + 5 * n - 2) / 16
        expected_4_cliques = (
            (a ** 6) / 24 *
            (   
                2 * E_n1_quad - 
                4 * n * E_n1_cube + 
                (6 * (n ** 2) - 18 * n + 22) * E_n1_square + 
                (-4 * (n ** 3) + 18 * (n ** 2) - 22 * n) * E_n1 + 
                (n ** 4) - 6 * (n ** 3) + 11 * (n ** 2) - 6 * n
            ) +
            (a ** 3) * (b ** 3) / 6 *
            (
                -2 * E_n1_quad +
                4 * n * E_n1_cube +
                (-3 * (n ** 2) + 3 * n - 4) * E_n1_square +
                ((n ** 3) - 3 * (n ** 2) + 4 * n) * E_n1
            ) +
            (a ** 2) * (b ** 4) / 4 *
            (
                E_n1_quad -
                2 * n * E_n1_cube + 
                ((n ** 2) + n - 1) * E_n1_square +
                (-(n ** 2) + n) * E_n1
            )
        )
        return expected_4_cliques
    else:
        raise Exception("q must be 3 or 4.")


In [3]:
print("Part 1 functions imported.")

Part 1 functions imported.
